In [1]:
import pandas as pd
import dask.dataframe as dd
from pathlib import Path

In [2]:
usage_df = pd.read_csv("/lcrc/project/FastBayes/rayandrew/alibaba-data-2017/container_usage.csv", names=[
    # "timestamp",
    # "event_type",
    # "isntance_id",
    # "machine_id",
    # "cpu_req",
    # "norm_mem_req",
    # "norm_disk_req",

    "time_stamp",
    "instance_id",
    "cpu_util_percent",
    "mem_util_percent",
    "disk_util_percent",
    "avg_cpu_1_min",
    "avg_cpu_5_min",
    "avg_cpu_15_min",
    "avg_cpi",
    "avg_cache_miss",
    "max_cpi",
    "max_cache_miss",
])
usage_df = usage_df.sort_values(by=["time_stamp"]).reset_index(drop=True)

In [3]:
usage_df.head()

,time_stamp,instance_id,cpu_util_percent,mem_util_percent,disk_util_percent,avg_cpu_1_min,avg_cpu_5_min,avg_cpu_15_min,avg_cpi,avg_cache_miss,max_cpi,max_cache_miss
0,39600,8978,13.18,69.460001,15.42,0.66,0.60,0.60,0.148605,0.196024,1.717350,2.671521
1,39600,9713,5.38,23.799999,7.10,0.60,0.50,0.44,0.120713,0.135784,1.657870,2.367960
2,39600,5952,1.50,34.960001,5.20,0.08,0.10,0.10,0.194770,0.266827,2.207530,3.900161
3,39600,7795,3.30,42.900002,16.00,0.22,0.22,0.20,0.183463,0.356302,3.043050,6.297808
4,39600,584,5.92,23.339999,7.50,0.54,0.52,0.50,0.123144,0.132585,1.716867,2.378259


In [4]:
usage_df[usage_df.instance_id == 107] 

,time_stamp,instance_id,cpu_util_percent,mem_util_percent,disk_util_percent,avg_cpu_1_min,avg_cpu_5_min,avg_cpu_15_min,avg_cpi,avg_cache_miss,max_cpi,max_cache_miss
8756,39600,107,3.34,24.080000,5.18,0.24,0.20,0.2,0.15543,0.550153,2.211467,12.187318
10545,39900,107,3.16,24.020000,5.20,0.22,0.20,0.2,0.15543,0.550153,2.211467,12.187318
21026,40200,107,3.28,24.000000,5.20,0.28,0.20,0.2,0.15543,0.550153,2.211467,12.187318
38592,40500,107,3.12,24.000000,5.14,0.22,0.22,0.2,0.15543,0.550153,2.211467,12.187318
42236,40800,107,3.22,24.000000,5.10,0.22,0.20,0.2,0.15543,0.550153,2.211467,12.187318
...,...,...,...,...,...,...,...,...,...,...,...,...
1434468,81300,107,2.66,24.200001,6.30,0.04,0.06,0.1,0.14174,0.563077,1.986490,12.448606
1445014,81600,107,2.90,24.160001,6.38,0.10,0.10,0.1,0.14174,0.563077,1.986490,12.448606
1456759,81900,107,2.66,24.200001,6.40,0.10,0.10,0.1,0.14174,0.563077,1.986490,12.448606
1460593,82200,107,2.84,24.180001,6.40,0.10,0.10,0.1,0.14174,0.563077,1.986490,12.448606


In [5]:
event_df = pd.read_csv("/lcrc/project/FastBayes/rayandrew/alibaba-data-2017/container_event.csv", names=[
    "time_stamp",
    "event_type",
    "instance_id",
    "machine_id",
    "plan_cpu",
    "plan_mem",
    "plan_disk",
    "cpu_set",
    "x"
])
event_df = event_df[event_df.columns[:-1]].reset_index(drop=True)

In [6]:
event_df.head()

,time_stamp,event_type,instance_id,machine_id,plan_cpu,plan_mem,plan_disk,cpu_set
0,0,Create,10000,934,4,0.042409,0.034085,40|41|42|43
1,0,Create,1000,1295,8,0.084819,0.056809,56|57|58|59|60|61|62|63
2,0,Create,10001,668,8,0.084819,0.056809,24|25|26|27|28|29|30|31
3,0,Create,10002,1217,8,0.084819,0.056809,4|5|6|7|8|9|10|11
4,0,Create,10003,1019,4,0.042409,0.034085,36|37|38|39


In [10]:
event_df[event_df.instance_id == 107]

,time_stamp,event_type,instance_id,machine_id,plan_cpu,plan_mem,plan_disk,cpu_set
765,0,Create,107,1063,4,0.042409,0.034085,40|41|42|43


In [21]:
event_df[event_df.instance_id == 7246]

,time_stamp,event_type,instance_id,machine_id,plan_cpu,plan_mem,plan_disk,cpu_set


In [ ]:
# len(event_df.instance_id.unique()) == len(event_df)

# for (instance_id, duplicated) in event_df.instance_id.duplicated(keep="last").items():
#     if duplicated:
#         print(instance_id)
#         print(len(event_df[event_df.instance_id == instance_id]))
#         print()
#         print()

In [ ]:
# event_df[event_df.instance_id == 9276]

In [ ]:
# event_df[event_df.instance_id == 831]

In [ ]:
# usage_df[usage_df.instance_id == 10000].cpu_util_percent.plot()

In [ ]:
# usage_df.groupby("instance_id").max()

In [ ]:
# def q50(x):
#     return x.quantile(0.5)

# def q90(x):
#     return x.quantile(0.9)

# def q95(x):
#     return x.quantile(0.95)

# def q99(x):
#     return x.quantile(0.99)

# grouped_df = usage_df.groupby(["instance_id"], as_index=False).agg({
#     'cpu_util_percent': [q50, q90, q95, q99, 'max', 'mean'],
# })
# grouped_df.columns = ["_".join(col_name).rstrip('_') for col_name in grouped_df.columns.to_flat_index()]
# grouped_df = grouped_df.reset_index(drop=True)

In [ ]:
# grouped_df["x"] = grouped_df.index
# grouped_df.head()

In [ ]:
# # grouped_df.iloc[:50].plot.scatter(y="cpu_util_percent_max", x="x", figsize=(20, 8))
# grouped_df.iloc[:125].plot(y="cpu_util_percent_mean", x="x", figsize=(20, 8))

In [16]:
merged_df = pd.merge(usage_df, event_df,  how='left', left_on=['instance_id'], right_on = ['instance_id'])
merged_df = merged_df.sort_values(by=["time_stamp_x", "time_stamp_y"]).reset_index(drop=True)
merged_df = merged_df.drop(columns=["time_stamp_y"])
merged_df = merged_df.rename(columns={"time_stamp_x": "time_stamp"})
merged_df = merged_df.reset_index(drop=True)
merged_df

,time_stamp,instance_id,cpu_util_percent,mem_util_percent,disk_util_percent,avg_cpu_1_min,avg_cpu_5_min,avg_cpu_15_min,avg_cpi,avg_cache_miss,max_cpi,max_cache_miss,event_type,machine_id,plan_cpu,plan_mem,plan_disk,cpu_set
0,39600,8978,13.18,69.460001,15.42,0.66,0.60,0.60,0.148605,0.196024,1.717350,2.671521,Create,669.0,4.0,0.042409,0.034085,52|53|54|55
1,39600,9713,5.38,23.799999,7.10,0.60,0.50,0.44,0.120713,0.135784,1.657870,2.367960,Create,686.0,8.0,0.169637,0.056809,0|1|2|3|4|5|6|7
2,39600,5952,1.50,34.960001,5.20,0.08,0.10,0.10,0.194770,0.266827,2.207530,3.900161,Create,859.0,4.0,0.042409,0.034085,28|29|30|31
3,39600,7795,3.30,42.900002,16.00,0.22,0.22,0.20,0.183463,0.356302,3.043050,6.297808,Create,387.0,4.0,0.042409,0.034085,8|9|10|11
4,39600,584,5.92,23.339999,7.50,0.54,0.52,0.50,0.123144,0.132585,1.716867,2.378259,Create,154.0,8.0,0.169637,0.056809,36|37|38|39|40|41|42|43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482623,82500,7246,6.20,50.820000,6.08,0.16,0.16,0.26,0.096272,0.108742,1.798856,2.170488,NaN,NaN,NaN,NaN,NaN,NaN
1482624,82500,5602,5.68,45.840000,12.30,0.56,0.58,0.60,0.076163,0.082030,1.321156,1.711819,NaN,NaN,NaN,NaN,NaN,NaN
1482625,82500,13,3.12,37.619999,4.90,0.10,0.10,0.10,0.123803,0.119564,1.806384,2.347003,NaN,NaN,NaN,NaN,NaN,NaN
1482626,82500,7859,5.08,25.319999,4.10,0.32,0.38,0.50,0.105509,0.093097,1.355383,1.474151,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
merged_df[merged_df.instance_id == 107]

,time_stamp,instance_id,cpu_util_percent,mem_util_percent,disk_util_percent,avg_cpu_1_min,avg_cpu_5_min,avg_cpu_15_min,avg_cpi,avg_cache_miss,max_cpi,max_cache_miss,event_type,machine_id,plan_cpu,plan_mem,plan_disk,cpu_set
8542,39600,107,3.34,24.080000,5.18,0.24,0.20,0.2,0.15543,0.550153,2.211467,12.187318,Create,1063.0,4.0,0.042409,0.034085,40|41|42|43
10550,39900,107,3.16,24.020000,5.20,0.22,0.20,0.2,0.15543,0.550153,2.211467,12.187318,Create,1063.0,4.0,0.042409,0.034085,40|41|42|43
21030,40200,107,3.28,24.000000,5.20,0.28,0.20,0.2,0.15543,0.550153,2.211467,12.187318,Create,1063.0,4.0,0.042409,0.034085,40|41|42|43
38426,40500,107,3.12,24.000000,5.14,0.22,0.22,0.2,0.15543,0.550153,2.211467,12.187318,Create,1063.0,4.0,0.042409,0.034085,40|41|42|43
42251,40800,107,3.22,24.000000,5.10,0.22,0.20,0.2,0.15543,0.550153,2.211467,12.187318,Create,1063.0,4.0,0.042409,0.034085,40|41|42|43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435955,81300,107,2.66,24.200001,6.30,0.04,0.06,0.1,0.14174,0.563077,1.986490,12.448606,Create,1063.0,4.0,0.042409,0.034085,40|41|42|43
1446513,81600,107,2.90,24.160001,6.38,0.10,0.10,0.1,0.14174,0.563077,1.986490,12.448606,Create,1063.0,4.0,0.042409,0.034085,40|41|42|43
1458230,81900,107,2.66,24.200001,6.40,0.10,0.10,0.1,0.14174,0.563077,1.986490,12.448606,Create,1063.0,4.0,0.042409,0.034085,40|41|42|43
1462276,82200,107,2.84,24.180001,6.40,0.10,0.10,0.1,0.14174,0.563077,1.986490,12.448606,Create,1063.0,4.0,0.042409,0.034085,40|41|42|43


In [18]:
# merged_df.iloc[:200].max_cpi.plot(figsize=(10, 4))

In [ ]:
# merged_df[merged_df.instance_id == 107].plan_disk.plot()

In [ ]:
# merged_df.groupby(["instance_id"], as_index=False).mean()

In [ ]:
# x_df = usage_df.groupby(["instance_id"], as_index=False)

In [ ]:
# x_df.quantile(.99).iloc[:150].cpu_util_percent.plot()

In [ ]:
# event_df[event_df.instance_id == 1]

In [19]:
merged_df[merged_df.instance_id == 1]

,time_stamp,instance_id,cpu_util_percent,mem_util_percent,disk_util_percent,avg_cpu_1_min,avg_cpu_5_min,avg_cpu_15_min,avg_cpi,avg_cache_miss,max_cpi,max_cache_miss,event_type,machine_id,plan_cpu,plan_mem,plan_disk,cpu_set
10189,39600,1,4.60,31.220000,3.5,0.26,0.28,0.30,0.155187,0.280369,2.552161,4.893782,NaN,NaN,NaN,NaN,NaN,NaN
20484,39900,1,4.42,31.260000,3.5,0.30,0.30,0.30,0.155187,0.280369,2.552161,4.893782,NaN,NaN,NaN,NaN,NaN,NaN
30762,40200,1,4.32,31.300000,3.5,0.36,0.34,0.30,0.155187,0.280369,2.552161,4.893782,NaN,NaN,NaN,NaN,NaN,NaN
41156,40500,1,4.06,31.320000,3.6,0.30,0.30,0.30,0.155187,0.280369,2.552161,4.893782,NaN,NaN,NaN,NaN,NaN,NaN
51416,40800,1,4.20,31.359999,3.6,0.36,0.34,0.30,0.155187,0.280369,2.552161,4.893782,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441051,81300,1,3.30,33.420001,4.1,0.24,0.30,0.40,0.111743,0.164048,2.155552,3.762658,NaN,NaN,NaN,NaN,NaN,NaN
1451420,81600,1,3.80,33.480000,4.1,0.18,0.24,0.34,0.111743,0.164048,2.155552,3.762658,NaN,NaN,NaN,NaN,NaN,NaN
1461792,81900,1,3.38,33.500000,4.1,0.22,0.20,0.30,0.111743,0.164048,2.155552,3.762658,NaN,NaN,NaN,NaN,NaN,NaN
1472141,82200,1,3.70,33.500000,4.1,0.24,0.22,0.30,0.111743,0.164048,2.155552,3.762658,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# len(merged_df)

In [20]:
merged_df.to_parquet("../raw_data/alibaba/container.parquet")